# Creating an OSKAR MWA telescope

In [10]:
from subprocess import call
from astropy.io import fits
import numpy as np
from pathlib import Path


First of all, create the telescope dir.

In [11]:
Path.mkdir(Path("MWA_phase1"), exist_ok=True)

Next, write the array centre longitude, latitude and height.

In [4]:
with open("MWA_phase1/position.txt", 'w') as outfile:
    outfile.write("116.67081523611111 -26.703319405555554 377.827\n")

Get the `enh` coords and tile names of the MWA phase 1 layout from a metafits file. Write the locations into the `layout.txt` file.

In [78]:
from wodenpy.wodenpy_setup.run_setup import get_antenna_order

##Ignore the dipamps part, this is a zenith pointing EoR0 field metafits
metafits = '../../../examples/metafits/1088285600_DipAmps.metafits'

with fits.open(metafits) as f:
    ##Need to order the antennas via the Tile column to be consistent
    ## with hyperdrive
    tileinds = f[1].data['Tile']
    
    ##to match hyperdrive, WODEN also reorders the antennas by tile name
    antenna_order = get_antenna_order(tileinds)

    ##metafits has positions for both X and Y pols which are identical so
    ##get every second position
    selection = np.arange(0, 256, 2)

    ##Get the east, north, height antenna positions from the metafits
    east = f[1].data['East'][antenna_order][selection]
    north = f[1].data['North'][antenna_order][selection]
    height = f[1].data['Height'][antenna_order][selection]
    tilenames = f[1].data['Tilename'][antenna_order][selection]
    orig_full_tile_names = f[1].data['Tilename']
    
print(tilenames)
# print(orig_tile_names)
    
with open("MWA_phase1/layout.txt", 'w') as outfile:
    for e, n, h in zip(east, north, height):
        outfile.write(f"{e:.12f} {n:.12f} {h:.12f}\n")


['Tile011' 'Tile012' 'Tile013' 'Tile014' 'Tile015' 'Tile016' 'Tile017'
 'Tile018' 'Tile021' 'Tile022' 'Tile023' 'Tile024' 'Tile025' 'Tile026'
 'Tile027' 'Tile028' 'Tile031' 'Tile032' 'Tile033' 'Tile034' 'Tile035'
 'Tile036' 'Tile037' 'Tile038' 'Tile041' 'Tile042' 'Tile043' 'Tile044'
 'Tile045' 'Tile046' 'Tile047' 'Tile048' 'Tile051' 'Tile052' 'Tile053'
 'Tile054' 'Tile055' 'Tile056' 'Tile057' 'Tile058' 'Tile061' 'Tile062'
 'Tile063' 'Tile064' 'Tile065' 'Tile066' 'Tile067' 'Tile068' 'Tile071'
 'Tile072' 'Tile073' 'Tile074' 'Tile075' 'Tile076' 'Tile077' 'Tile078'
 'Tile081' 'Tile082' 'Tile083' 'Tile084' 'Tile085' 'Tile086' 'Tile087'
 'Tile088' 'Tile091' 'Tile092' 'Tile093' 'Tile094' 'Tile095' 'Tile096'
 'Tile097' 'Tile098' 'Tile101' 'Tile102' 'Tile103' 'Tile104' 'Tile105'
 'Tile106' 'Tile107' 'Tile108' 'Tile111' 'Tile112' 'Tile113' 'Tile114'
 'Tile115' 'Tile116' 'Tile117' 'Tile118' 'Tile121' 'Tile122' 'Tile123'
 'Tile124' 'Tile125' 'Tile126' 'Tile127' 'Tile128' 'Tile131' 'Tile132'
 'Tile

Finally, we need to detail the station layout, which is an MWA tile. We could do this with a telescope-wide `station/layout.txt` file, but to link positions to tiles, I'll create stations named after each tile. TODO infact, this could be a hack to do missing dipoles/dipole amplitudes in the future?

In [29]:
##Spacing between dipoles is 1.1m on the ground mesh.
##Station layout coords are relative to station centre
##Farthest dipole centre from station centre is then (3*1.1)/2 = 1.65
##Assume flat layout (height = 0)


for tile in tilenames:
    Path.mkdir(Path(f"MWA_phase1/{tile}"), exist_ok=True)

    east_start = -1.65
    north_start = -1.65

    #os.mkdir('station')
    out_file = open(f'MWA_phase1/{tile}/layout.txt','w+')

    for i in range(4):
        for j in range(4):
            out_file.write('%.3f %.3f\n' %(east_start,north_start))
            east_start += 1.1
        east_start = -1.65
        north_start += 1.1

    out_file.close()

Make a really simple single point source at phase flat spectrum sky model.

In [30]:
ra = 0.0
dec = -26.703319405555554
flux_I = 1.0
flux_Q = 0.0
flux_U = 0.0
flux_V = 0.0
ref_freq = 180e6
SI = 0.0

with open("skymodel.osm", 'w') as outfile:
    outfile.write(f"{ra} {dec} {flux_I} {flux_Q} {flux_U} {flux_V} {ref_freq} {SI}\n")


Now we have the telescope and sky model, we can run `OSKAR`. Obviously this requires `OSKAR` to be installed. I've written a settings file `run_oskar_sim_interferometer.ini`, which I run via

```bash
$ oskar_sim_interferometer run_oskar_sim_interferometer.ini

```

## Make a dipole flagged version

In [36]:
Path.mkdir(Path("MWA_phase1_flags"), exist_ok=True)
with open("MWA_phase1_flags/position.txt", 'w') as outfile:
    outfile.write("116.67081523611111 -26.703319405555554 377.827\n")
    
with open("MWA_phase1_flags/layout.txt", 'w') as outfile:
    for e, n, h in zip(east, north, height):
        outfile.write(f"{e:.12f} {n:.12f} {h:.12f}\n")


In [76]:
metafits = '../../../examples/metafits/1088285600_DipAmps.metafits'

##Insert some dipole flags into the metafits (just flag the first 16 for now,
# one for each dipole)
with fits.open(metafits) as f:
    ##Need to order the antennas via the Tile column to be consistent
    ## with hyperdrive
    
    delays = f[1].data['Delays']
    
    ##cos of the stupid re-ordering of the antennas by antenna name, find
    ##what the actual first 16 antennas are
    first_16 = antenna_order[:32]
    
    for dipole in range(16):
        
        x_dip = dipole*2
        y_dip = x_dip + 1
        
        print(orig_full_tile_names[first_16[x_dip]], orig_full_tile_names[first_16[y_dip]])
        
        delays[first_16[x_dip]][dipole] = 32
        delays[first_16[y_dip]][dipole] = 32
    
    f[1].data['Delays'] = delays
    
    ##Write this out, which means we can use it internally to WODEN and hyperbeam
    f.writeto('1088285600_flags.metafits', overwrite=True)
    
# for i in range(128):
#     print(delays[2*i], delays[2*i + 1])

[86 87 84 85 82 83 80 81 94 95 92 93 90 91 88 89 70 71 68 69 66 67 64 65
 78 79 76 77 74 75 72 73]
Tile011 Tile011
Tile012 Tile012
Tile013 Tile013
Tile014 Tile014
Tile015 Tile015
Tile016 Tile016
Tile017 Tile017
Tile018 Tile018
Tile021 Tile021
Tile022 Tile022
Tile023 Tile023
Tile024 Tile024
Tile025 Tile025
Tile026 Tile026
Tile027 Tile027
Tile028 Tile028


Now go through writing out each station layout, but if there is a flag, leave out the dipole.

In [83]:
reorder_delays = delays[antenna_order, :]
reorder_delays = reorder_delays[selection, :]

flag_tile_names = []

for tile_ind, tile in enumerate(tilenames):
    Path.mkdir(Path(f"MWA_phase1_flags/{tile}"), exist_ok=True)

    east_start = -1.65
    north_start = -1.65

    #os.mkdir('station')
    out_file = open(f'MWA_phase1_flags/{tile}/layout.txt','w+')

    for i in range(4):
        for j in range(4):
            if reorder_delays[tile_ind][i*4+j] == 32:
                print(tile_ind, tilenames[tile_ind])
                flag_tile_names.append(tilenames[tile_ind])
                pass
            else:
                out_file.write('%.3f %.3f\n' %(east_start,north_start))
            east_start += 1.1
        east_start = -1.65
        north_start += 1.1

    out_file.close()

0 Tile011
1 Tile012
2 Tile013
3 Tile014
4 Tile015
5 Tile016
6 Tile017
7 Tile018
8 Tile021
9 Tile022
10 Tile023
11 Tile024
12 Tile025
13 Tile026
14 Tile027
15 Tile028


In [69]:

np.savetxt('MWA_phase1_flags/station_type_map.txt', range(128), fmt='%d')
